<a href="https://colab.research.google.com/github/jpupkies/Jim-Pupkies/blob/master/Gemini_Integrated_Portfolio_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini Integrated Portfolio Insights

### Purpose
This notebook demonstrates a **comprehensive, AI-assisted portfolio analysis pipeline**. It is designed to provide **investors and analysts with actionable insights** while showcasing advanced **data engineering, visualization, and AI reasoning skills**.

---

### Workflow Overview
1. **Portfolio Data Loading & Cleaning**  
   - Load current portfolio positions with market value, allocation %, gain/loss %, and cost basis.  
   - Prepare data for analysis and visualization.

2. **Portfolio Visualization**  
   - Interactive **allocation donut chart**.  
   - **Gain/Loss scatter plots** and **bubble charts** for AI-assisted recommendations.  
   - Multi-step **rebalancing suggestion scatter chart**.

3. **AI-Assisted Insights**  
   - Detect high concentration risk (>20% allocation).  
   - Identify top performers and underperformers.  
   - Generate actionable AI recommendations with **confidence scores**.  

4. **Multi-Step Reasoning & Rule-Grounded Rebalancing**  
   - Combine AI recommendations with simple financial rules:  
     - Reduce positions with high allocation  
     - Monitor or consider selling underperformers  
     - Consider taking profit on extreme gains  
     - Limit exposure for positions >25% of portfolio  

5. **Professional Reporting**  
   - Inline, interactive charts in Colab.  
   - Summary table with **conditional formatting** for clear visualization of recommendations.  
   - Export **downloadable HTML report** for offline review and presentation.  

---

### Skills Demonstrated
- **Data Engineering & Cleaning**: Transforming portfolio data for analysis.  
- **Interactive Visualization**: Plotly charts with dynamic sizing and color coding.  
- **Applied AI & Multi-Step Reasoning**: Generating actionable portfolio insights with confidence-aware recommendations.  
- **Financial Logic Integration**: Combining AI output with portfolio risk and rebalancing rules.  
- **Reporting & Automation**: Conditional formatting, HTML export, and downloadable interactive reports.  

---

This notebook is designed as a **portfolio-ready demonstration** of full-stack **data, AI, and finance engineering capabilities** suitable for potential employers or investors.

In [1]:
# ==============================
# Environment Setup
# Gemini_Integrated_Portfolio_Insights
# ==============================

# Data manipulation & analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Optional: financial data APIs (e.g., yfinance)
# import yfinance as yf

# AI / Language model integration
# from openai import OpenAI
# client = OpenAI(api_key="YOUR_OPENAI_API_KEY")

# Utilities
import datetime
import warnings
warnings.filterwarnings("ignore")

# ==============================
# Helper Functions (Optional)
# ==============================

def compute_market_value(quantity, price):
    """Compute market value for a given position."""
    return quantity * price

def compute_gain_loss(market_value, cost_basis):
    """Compute gain/loss in dollars and percent."""
    gain_loss_dollar = market_value - cost_basis
    gain_loss_percent = (gain_loss_dollar / cost_basis) * 100 if cost_basis != 0 else 0
    return gain_loss_dollar, gain_loss_percent

✅ **What this does**:

- Sets up all core libraries for data analysis and plotting.

- Prepares helper functions for key financial calculations.

- Includes placeholders for AI and API integration.

In [2]:
# ==============================
# Load Portfolio Data & Compute Metrics
# ==============================

# Example: Portfolio data (replace with CSV/JSON if preferred)
portfolio_data = [
    {"Symbol": "WBD", "Name": "WARNER BROS DISCOVERY INTENDER OFFER EXP: 01/21/26", "Quantity": 31, "Price": 28.82, "Cost_Basis": 381.71},
    {"Symbol": "COHR", "Name": "COHERENT CORP", "Quantity": 3, "Price": 184.57, "Cost_Basis": 292.89},
    {"Symbol": "VSCO", "Name": "VICTORIAS SECRET & CO", "Quantity": 5, "Price": 54.17, "Cost_Basis": 157.20},
    {"Symbol": "PHAT", "Name": "PHATHOM PHARMACEUTICALS", "Quantity": 10, "Price": 16.59, "Cost_Basis": 102.16},
    {"Symbol": "APGE", "Name": "APOGEE THERAPEUTICS INC", "Quantity": 3, "Price": 75.48, "Cost_Basis": 202.23},
    {"Symbol": "VICR", "Name": "VICOR CORP", "Quantity": 3, "Price": 109.60, "Cost_Basis": 294.33},
    {"Symbol": "WDC", "Name": "WESTERN DIGITAL CORP", "Quantity": 11, "Price": 172.27, "Cost_Basis": 1702.29},
    {"Symbol": "LLY", "Name": "ELI LILLY AND CO", "Quantity": 1, "Price": 1074.68, "Cost_Basis": 966.55},
    {"Symbol": "ANAB", "Name": "ANAPTYSBIO INC", "Quantity": 4, "Price": 48.48, "Cost_Basis": 181.40},
    {"Symbol": "CSCO", "Name": "CISCO SYS INC", "Quantity": 18, "Price": 76.62, "Cost_Basis": 1329.40},
    {"Symbol": "THR", "Name": "THERMON GROUP HLDGS INC", "Quantity": 5, "Price": 37.16, "Cost_Basis": 179.45},
    {"Symbol": "NRC", "Name": "NTNL RESH CORP", "Quantity": 60, "Price": 18.77, "Cost_Basis": 1127.40},
    {"Symbol": "ANRO", "Name": "ALTO NEUROSCIENCE IN", "Quantity": 17, "Price": 17.80, "Cost_Basis": 304.64},
    {"Symbol": "BTE", "Name": "BAYTEX ENERGY CORP F", "Quantity": 220, "Price": 3.23, "Cost_Basis": 719.40},
    {"Symbol": "FLS", "Name": "FLOWSERVE CORP", "Quantity": 20, "Price": 69.38, "Cost_Basis": 1421.00},
    {"Symbol": "HAE", "Name": "HAEMONETICS CORP MASS", "Quantity": 3, "Price": 80.15, "Cost_Basis": 247.88}
]

# Convert to DataFrame
portfolio_df = pd.DataFrame(portfolio_data)

# Compute derived metrics
portfolio_df['Market_Value'] = portfolio_df.apply(lambda row: compute_market_value(row['Quantity'], row['Price']), axis=1)
portfolio_df['Gain_Loss_$'], portfolio_df['Gain_Loss_%'] = zip(*portfolio_df.apply(
    lambda row: compute_gain_loss(row['Market_Value'], row['Cost_Basis']), axis=1))

# Compute allocation %
total_value = portfolio_df['Market_Value'].sum()
portfolio_df['Allocation_%'] = (portfolio_df['Market_Value'] / total_value) * 100

# Display the DataFrame
portfolio_df = portfolio_df.sort_values(by='Market_Value', ascending=False).reset_index(drop=True)
portfolio_df

,Symbol,Name,Quantity,Price,Cost_Basis,Market_Value,Gain_Loss_$,Gain_Loss_%,Allocation_%
0,WDC,WESTERN DIGITAL CORP,11,172.27,1702.29,1894.97,192.68,11.318870,17.329243
1,FLS,FLOWSERVE CORP,20,69.38,1421.00,1387.60,-33.40,-2.350457,12.689413
2,CSCO,CISCO SYS INC,18,76.62,1329.40,1379.16,49.76,3.743042,12.612230
3,NRC,NTNL RESH CORP,60,18.77,1127.40,1126.20,-1.20,-0.106440,10.298946
4,LLY,ELI LILLY AND CO,1,1074.68,966.55,1074.68,108.13,11.187212,9.827802
5,WBD,WARNER BROS DISCOVERY INTENDER OFFER EXP: 01/2...,31,28.82,381.71,893.42,511.71,134.057269,8.170204
6,BTE,BAYTEX ENERGY CORP F,220,3.23,719.40,710.60,-8.80,-1.223242,6.498340
7,COHR,COHERENT CORP,3,184.57,292.89,553.71,260.82,89.050497,5.063603
8,VICR,VICOR CORP,3,109.60,294.33,328.80,34.47,11.711344,3.006831
9,ANRO,ALTO NEUROSCIENCE IN,17,17.80,304.64,302.60,-2.04,-0.669643,2.767236


✅ **What this does**:

1. Loads portfolio positions into a DataFrame.

2. Computes Market Value = Quantity × Price.

3. Computes Gain/Loss $ and Gain/Loss %.

4. Computes Allocation % of the total portfolio for each position.

5. Sorts the DataFrame by Market Value for easier analysis.

In [7]:
# ==============================
# Enhanced Exploratory Data Analysis (EDA)
# ==============================

# Set a consistent Plotly template
plotly_template = "plotly_white"

# ------------------------------
# 1. Portfolio Allocation Pie Chart
# ------------------------------
fig_allocation = px.pie(
    portfolio_df,
    names='Symbol',
    values='Allocation_%',
    title='Portfolio Allocation by Position',
    hover_data=['Name', 'Market_Value', 'Gain_Loss_%'],
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig_allocation.update_traces(
    textposition='inside',
    textinfo='percent+label',
    pull=[0.05 if val > 15 else 0 for val in portfolio_df['Allocation_%']]  # Pull out large positions
)
fig_allocation.update_layout(
    title_font_size=20,
    legend_title="Symbols",
    legend=dict(orientation="h", y=-0.1, x=0.5, xanchor='center')
)
fig_allocation.show(template=plotly_template)

# ------------------------------
# 2. Top Performers & Underperformers
# ------------------------------
top_gainers = portfolio_df.sort_values(by='Gain_Loss_%', ascending=False).head(5)
top_losers = portfolio_df.sort_values(by='Gain_Loss_%', ascending=True).head(5)

print("### Top 5 Performers (% Gain) ###")
display(top_gainers[['Symbol', 'Name', 'Gain_Loss_%', 'Market_Value']])

print("### Top 5 Underperformers (% Loss) ###")
display(top_losers[['Symbol', 'Name', 'Gain_Loss_%', 'Market_Value']])

# ------------------------------
# 3. Gain/Loss % Bar Chart
# ------------------------------
fig_gain_loss = px.bar(
    portfolio_df,
    x='Symbol',
    y='Gain_Loss_%',
    color='Gain_Loss_%',
    color_continuous_scale=px.colors.diverging.RdYlGn,
    title='Gain/Loss % by Position',
    hover_data=['Name', 'Market_Value', 'Allocation_%']
)
fig_gain_loss.update_layout(
    template=plotly_template,
    yaxis_title='Gain/Loss (%)',
    xaxis_title='Symbol',
    title_font_size=20,
    coloraxis_colorbar=dict(title="Gain/Loss %")
)
fig_gain_loss.update_traces(marker_line_color='black', marker_line_width=1.5)
fig_gain_loss.show()

# ------------------------------
# 4. Market Value Distribution Histogram
# ------------------------------
fig_market_value = px.histogram(
    portfolio_df,
    x='Market_Value',
    nbins=8,
    title='Distribution of Market Value Across Positions',
    color_discrete_sequence=['#636EFA'],
    marginal='box',  # Adds a box plot on top for distribution
    hover_data=['Symbol', 'Gain_Loss_%', 'Allocation_%']
)
fig_market_value.update_layout(
    template=plotly_template,
    title_font_size=20,
    xaxis_title='Market Value ($)',
    yaxis_title='Number of Positions'
)
fig_market_value.show()

### Top 5 Performers (% Gain) ###


,Symbol,Name,Gain_Loss_%,Market_Value
5,WBD,WARNER BROS DISCOVERY INTENDER OFFER EXP: 01/2...,134.057269,893.42
7,COHR,COHERENT CORP,89.050497,553.71
10,VSCO,VICTORIAS SECRET & CO,72.296438,270.85
15,PHAT,PHATHOM PHARMACEUTICALS,62.392326,165.90
12,APGE,APOGEE THERAPEUTICS INC,11.971518,226.44


### Top 5 Underperformers (% Loss) ###


,Symbol,Name,Gain_Loss_%,Market_Value
11,HAE,HAEMONETICS CORP MASS,-2.997418,240.45
1,FLS,FLOWSERVE CORP,-2.350457,1387.60
6,BTE,BAYTEX ENERGY CORP F,-1.223242,710.60
9,ANRO,ALTO NEUROSCIENCE IN,-0.669643,302.60
3,NRC,NTNL RESH CORP,-0.106440,1126.20


✅ **What this EDA section does**:

1. Portfolio Allocation Pie Chart:

- Shows how each position contributes to the total portfolio.

- Highlights concentration in certain stocks.
<br>
2. Top Performers & Underperformers Table:

- Quickly identifies best and worst performing positions.
<br>
3. Gain/Loss % Bar Chart:

- Visualizes performance per position with intuitive color scaling (green = gain, red = loss).
<br>
4. Market Value Distribution:

- Helps spot whether the portfolio is heavily weighted in a few large positions.

In [8]:
# ==============================
# AI-Assisted Portfolio Insights
# ==============================

# Optional: Define a simple function to generate insights using a language model
# Replace `ai_model_generate` with Gemini/OpenAI call
def ai_model_generate(prompt: str) -> str:
    """
    Simulated AI function. Replace with actual API call.
    """
    # For demo purposes, we return a placeholder response
    return f"AI Insight based on portfolio data:\n{prompt[:200]}..."

# Step 1: Identify concentration risk (>20% in a single position)
high_concentration = portfolio_df[portfolio_df['Allocation_%'] > 20]

# Step 2: Identify top gainers and laggards
top_gainers = portfolio_df.sort_values('Gain_Loss_%', ascending=False).head(3)
top_losers = portfolio_df.sort_values('Gain_Loss_%', ascending=True).head(3)

# Step 3: Construct AI prompt
ai_prompt = f"""
Portfolio Summary:
Total Positions: {len(portfolio_df)}
Total Portfolio Value: ${portfolio_df['Market_Value'].sum():,.2f}

High Concentration Risk Positions (>20% allocation):
{high_concentration[['Symbol', 'Allocation_%']].to_dict(orient='records')}

Top 3 Gainers:
{top_gainers[['Symbol', 'Gain_Loss_%', 'Market_Value']].to_dict(orient='records')}

Top 3 Underperformers:
{top_losers[['Symbol', 'Gain_Loss_%', 'Market_Value']].to_dict(orient='records')}

Provide actionable insights, potential rebalancing suggestions, and highlight positions with risk.
"""

# Step 4: Generate AI insights (replace this with Gemini/OpenAI API call)
portfolio_insights = ai_model_generate(ai_prompt)

# Display AI Insights
print("### AI-Assisted Portfolio Insights ###\n")
print(portfolio_insights)

### AI-Assisted Portfolio Insights ###

AI Insight based on portfolio data:

Portfolio Summary:
Total Positions: 16
Total Portfolio Value: $10,935.10

High Concentration Risk Positions (>20% allocation):
[]

Top 3 Gainers:
[{'Symbol': 'WBD', 'Gain_Loss_%': 134.05726860705772,...


✅ **What this section does**

1. Concentration Risk Detection

- Flags positions >20% allocation that could pose portfolio risk.
<br>
2. Performance Highlights

- Top 3 gainers and top 3 underperformers are automatically pulled.
<br>
3. AI Reasoning

- Combines portfolio metrics into a prompt for the AI model.

- Generates actionable insights, like monitoring, holding, or rebalancing.

- Can include confidence or abstention notes if using a model that supports it.
<br>
4. Flexible

- You can replace the ai_model_generate placeholder with Gemini/OpenAI API calls, and even add multi-step reasoning, like:

In [10]:
if len(high_concentration) > 0:
    # Ask AI specifically to suggest rebalancing
    pass  # placeholder to prevent SyntaxError

In [11]:
# ==============================
# Enhanced AI-Assisted Portfolio Insights
# ==============================

import random

# ------------------------------
# Step 1: Detect Concentration Risk
# ------------------------------
high_concentration = portfolio_df[portfolio_df['Allocation_%'] > 20]

# ------------------------------
# Step 2: Identify Performance Extremes
# ------------------------------
top_gainers = portfolio_df.sort_values('Gain_Loss_%', ascending=False).head(3)
top_losers = portfolio_df.sort_values('Gain_Loss_%', ascending=True).head(3)

# ------------------------------
# Step 3: Generate AI Recommendations (Simulated)
# ------------------------------
def generate_recommendation(row):
    """
    Simulated AI recommendation for portfolio position.
    Replace this with Gemini/OpenAI API call in production.
    """
    # Basic rules for demo purposes
    if row['Allocation_%'] > 20:
        action = "Reduce"
    elif row['Gain_Loss_%'] > 50:
        action = "Hold / Consider Taking Profit"
    elif row['Gain_Loss_%'] < 0:
        action = "Monitor / Potential Sell"
    else:
        action = "Hold"

    # Simulated confidence score (0-1)
    confidence = round(random.uniform(0.7, 0.99), 2)

    return pd.Series([action, confidence])

# Apply AI recommendations
portfolio_df[['AI_Recommendation', 'Confidence']] = portfolio_df.apply(generate_recommendation, axis=1)

# ------------------------------
# Step 4: Color-Coding Recommendations
# ------------------------------
def recommendation_color(action):
    colors = {
        "Reduce": "#FF6F61",             # Red / Urgent
        "Hold / Consider Taking Profit": "#FFA500",  # Orange
        "Monitor / Potential Sell": "#FFD700",       # Yellow
        "Hold": "#4CAF50"                # Green / Safe
    }
    return colors.get(action, "#B0B0B0")  # Default gray

portfolio_df['Color'] = portfolio_df['AI_Recommendation'].apply(recommendation_color)

# ------------------------------
# Step 5: Display Recommendations Table
# ------------------------------
import plotly.express as px

fig_recommendations = px.scatter(
    portfolio_df,
    x='Allocation_%',
    y='Gain_Loss_%',
    color='AI_Recommendation',
    size='Market_Value',
    hover_data=['Symbol', 'Name', 'Market_Value', 'Gain_Loss_%', 'Confidence'],
    title='AI-Assisted Portfolio Recommendations',
    color_discrete_map={
        "Reduce": "#FF6F61",
        "Hold / Consider Taking Profit": "#FFA500",
        "Monitor / Potential Sell": "#FFD700",
        "Hold": "#4CAF50"
    },
    template='plotly_white',
    size_max=50
)

fig_recommendations.update_layout(
    xaxis_title='Allocation (%)',
    yaxis_title='Gain/Loss (%)',
    title_font=dict(size=22, family='Arial Black')
)

fig_recommendations.show()

# ------------------------------
# Step 6: Summary Outputs
# ------------------------------
print("### High Concentration Risk Positions (>20% allocation) ###")
if not high_concentration.empty:
    display(high_concentration[['Symbol', 'Name', 'Allocation_%', 'Market_Value']])
else:
    print("None")

print("\n### Top AI Recommendations ###")
display(portfolio_df[['Symbol', 'Name', 'AI_Recommendation', 'Confidence', 'Market_Value']].sort_values('Confidence', ascending=False))

### High Concentration Risk Positions (>20% allocation) ###
None

### Top AI Recommendations ###


,Symbol,Name,AI_Recommendation,Confidence,Market_Value
14,THR,THERMON GROUP HLDGS INC,Hold,0.98,185.80
15,PHAT,PHATHOM PHARMACEUTICALS,Hold / Consider Taking Profit,0.97,165.90
6,BTE,BAYTEX ENERGY CORP F,Monitor / Potential Sell,0.95,710.60
5,WBD,WARNER BROS DISCOVERY INTENDER OFFER EXP: 01/2...,Hold / Consider Taking Profit,0.94,893.42
4,LLY,ELI LILLY AND CO,Hold,0.92,1074.68
12,APGE,APOGEE THERAPEUTICS INC,Hold,0.90,226.44
3,NRC,NTNL RESH CORP,Monitor / Potential Sell,0.90,1126.20
9,ANRO,ALTO NEUROSCIENCE IN,Monitor / Potential Sell,0.86,302.60
10,VSCO,VICTORIAS SECRET & CO,Hold / Consider Taking Profit,0.83,270.85
7,COHR,COHERENT CORP,Hold / Consider Taking Profit,0.83,553.71


✅ **Enhancements in This Section**

1. Position-by-position AI recommendations

- Assigns “Hold”, “Reduce”, “Monitor”, or “Hold/Take Profit” to each position
<br>
2. Color-coded scatter plot

- Visualizes allocation vs gain/loss with bubble size representing market value

- Colors correspond to recommendation category
<br>
3. Confidence score

- Each recommendation includes a simulated confidence (0–1)

- Can be replaced with model-generated confidence when using Gemini/OpenAI
<br>
4. Summary tables

- High concentration risk positions

- Full portfolio with recommendations, confidence, and market value
<br>
5. Professional, interactive Plotly visualization

- Perfect for demonstrating multi-step reasoning, AI integration, and data visualization skills

In [12]:
# ==============================
# Multi-Step AI & Rule-Grounded Rebalancing Suggestions
# ==============================

def generate_rebalancing_suggestion(row, total_portfolio_value):
    """
    Combines AI recommendation with financial rules to suggest portfolio actions.
    """
    suggestion = ""
    reason = ""

    # Step 1: High concentration check
    if row['Allocation_%'] > 20:
        suggestion = "Reduce Position"
        reason = f"High allocation ({row['Allocation_%']:.1f}%)"

    # Step 2: Loss monitoring
    elif row['Gain_Loss_%'] < -5:
        if suggestion == "":
            suggestion = "Monitor / Consider Sell"
        reason = f"Underperforming (-{abs(row['Gain_Loss_%']):.1f}%)"

    # Step 3: Extreme gain taking
    elif row['Gain_Loss_%'] > 50:
        if suggestion == "":
            suggestion = "Consider Taking Profit"
        reason = f"High gain (+{row['Gain_Loss_%']:.1f}%)"

    # Step 4: AI recommendation fallback
    if suggestion == "":
        suggestion = row['AI_Recommendation']
        reason = "AI-guided recommendation"

    # Step 5: Position sizing recommendation
    position_pct = row['Market_Value'] / total_portfolio_value * 100
    if position_pct > 25 and suggestion not in ["Reduce Position"]:
        suggestion += " / Limit Exposure"
        reason += " | Position >25% of portfolio"

    return pd.Series([suggestion, reason])

# Apply multi-step rebalancing logic
total_portfolio_value = portfolio_df['Market_Value'].sum()
portfolio_df[['Rebalancing_Suggestion', 'Reason']] = portfolio_df.apply(
    lambda row: generate_rebalancing_suggestion(row, total_portfolio_value),
    axis=1
)

# ------------------------------
# Display Enhanced Recommendations
# ------------------------------
import plotly.express as px

fig_rebalancing = px.scatter(
    portfolio_df,
    x='Allocation_%',
    y='Gain_Loss_%',
    color='Rebalancing_Suggestion',
    size='Market_Value',
    hover_data=['Symbol', 'Name', 'Market_Value', 'Gain_Loss_%', 'Confidence', 'Reason'],
    title='Multi-Step Portfolio Rebalancing Suggestions',
    template='plotly_white',
    size_max=50
)

# Color map for clarity
color_map = {
    "Reduce Position": "#FF6F61",
    "Monitor / Consider Sell": "#FFD700",
    "Consider Taking Profit": "#FFA500",
    "Hold": "#4CAF50",
    "Hold / Limit Exposure": "#4CAF50"
}
fig_rebalancing.update_traces(marker=dict(line=dict(width=1, color='black')))
fig_rebalancing.update_layout(
    title_font=dict(size=22, family='Arial Black'),
    xaxis_title='Allocation (%)',
    yaxis_title='Gain/Loss (%)'
)
fig_rebalancing.show()

# ------------------------------
# Summary Table
# ------------------------------
print("### Multi-Step Rebalancing Suggestions ###")
display(portfolio_df[['Symbol', 'Name', 'Rebalancing_Suggestion', 'Reason', 'Allocation_%', 'Gain_Loss_%', 'Market_Value']].sort_values('Allocation_%', ascending=False))

### Multi-Step Rebalancing Suggestions ###


,Symbol,Name,Rebalancing_Suggestion,Reason,Allocation_%,Gain_Loss_%,Market_Value
0,WDC,WESTERN DIGITAL CORP,Hold,AI-guided recommendation,17.329243,11.318870,1894.97
1,FLS,FLOWSERVE CORP,Monitor / Potential Sell,AI-guided recommendation,12.689413,-2.350457,1387.60
2,CSCO,CISCO SYS INC,Hold,AI-guided recommendation,12.612230,3.743042,1379.16
3,NRC,NTNL RESH CORP,Monitor / Potential Sell,AI-guided recommendation,10.298946,-0.106440,1126.20
4,LLY,ELI LILLY AND CO,Hold,AI-guided recommendation,9.827802,11.187212,1074.68
5,WBD,WARNER BROS DISCOVERY INTENDER OFFER EXP: 01/2...,Consider Taking Profit,High gain (+134.1%),8.170204,134.057269,893.42
6,BTE,BAYTEX ENERGY CORP F,Monitor / Potential Sell,AI-guided recommendation,6.498340,-1.223242,710.60
7,COHR,COHERENT CORP,Consider Taking Profit,High gain (+89.1%),5.063603,89.050497,553.71
8,VICR,VICOR CORP,Hold,AI-guided recommendation,3.006831,11.711344,328.80
9,ANRO,ALTO NEUROSCIENCE IN,Monitor / Potential Sell,AI-guided recommendation,2.767236,-0.669643,302.60


✅ **Enhancements Added**

1. Multi-Step Reasoning Pipeline

- Step 1: High concentration → reduce

- Step 2: Large losses → monitor/sell

- Step 3: Extreme gains → take profit

- Step 4: AI recommendation as fallback

- Step 5: Position sizing check (>25% → limit exposure)
<br>
2. Interactive Scatter Plot

- Color-coded rebalancing suggestions

- Bubble size = market value

- Hover info includes AI confidence and reasoning
<br>
3. Professional Summary Table

- Shows reasoning behind each suggestion

- Includes allocation %, gain/loss %, and market value
<br>
4. Showcases Skills for Employers

- Data engineering: cleaning, metrics

- Visualization & interactivity: Plotly charts

- Applied AI reasoning: multi-step, confidence-aware suggestions

- Financial logic integration: rebalancing rules

In [15]:
# ==============================
# Professional Portfolio Reporting for Colab
# ==============================

from IPython.display import display, HTML
from google.colab import files
import plotly.io as pio
from datetime import datetime
import os

# Create folder for saved reports
report_folder = "Portfolio_Reports"
os.makedirs(report_folder, exist_ok=True)

# Timestamped HTML filename
report_filename = f"Portfolio_Report_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.html"
report_path = os.path.join(report_folder, report_filename)

# ------------------------------
# Step 1: Display Charts Inline
# ------------------------------
print("### Portfolio Allocation ###")
fig_allocation.show()

print("### AI-Assisted Recommendations ###")
fig_recommendations.show()

print("### Multi-Step Rebalancing Suggestions ###")
fig_rebalancing.show()

# ------------------------------
# Step 2: Display Summary Table Inline
# ------------------------------
print("### Portfolio Summary Table ###")
display(portfolio_df[['Symbol', 'Name', 'Allocation_%', 'Gain_Loss_%',
                      'Market_Value', 'AI_Recommendation', 'Confidence',
                      'Rebalancing_Suggestion', 'Reason']])

# ------------------------------
# Step 3: Generate Downloadable HTML Report
# ------------------------------

# Convert summary table to HTML
html_summary = portfolio_df[['Symbol', 'Name', 'Allocation_%', 'Gain_Loss_%',
                             'Market_Value', 'AI_Recommendation', 'Confidence',
                             'Rebalancing_Suggestion', 'Reason']].to_html(
    index=False,
    classes='table table-striped table-hover',
    justify='center'
)

# Combine charts and table into full HTML report
# (Charts are saved separately as standalone HTML files)
pio.write_html(fig_allocation, file=os.path.join(report_folder, "Allocation_Chart.html"), auto_open=False)
pio.write_html(fig_recommendations, file=os.path.join(report_folder, "AI_Recommendations_Chart.html"), auto_open=False)
pio.write_html(fig_rebalancing, file=os.path.join(report_folder, "Rebalancing_Chart.html"), auto_open=False)

full_html = f"""
<html>
<head>
    <title>Portfolio Report - {datetime.now().strftime('%Y-%m-%d')}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 30px; }}
        h1 {{ color: #333333; }}
        h2 {{ color: #444444; margin-top: 30px; }}
        .table {{ border-collapse: collapse; width: 100%; }}
        .table th, .table td {{ border: 1px solid #ddd; padding: 8px; text-align: center; }}
        .table th {{ background-color: #f2f2f2; }}
    </style>
</head>
<body>
    <h1>Gemini Integrated Portfolio Insights</h1>
    <h2>Summary Table</h2>
    {html_summary}
    <p>Interactive charts saved separately in the folder: {report_folder}</p>
</body>
</html>
"""

# Save full HTML report
with open(report_path, "w") as f:
    f.write(full_html)

# ------------------------------
# Step 4: Provide Download Link
# ------------------------------
print(f"✅ Portfolio report saved: {report_path}")
print("Click the link below to download the HTML report:")
files.download(report_path)

### Portfolio Allocation ###


### AI-Assisted Recommendations ###


### Multi-Step Rebalancing Suggestions ###


### Portfolio Summary Table ###


,Symbol,Name,Allocation_%,Gain_Loss_%,Market_Value,AI_Recommendation,Confidence,Rebalancing_Suggestion,Reason
0,WDC,WESTERN DIGITAL CORP,17.329243,11.318870,1894.97,Hold,0.81,Hold,AI-guided recommendation
1,FLS,FLOWSERVE CORP,12.689413,-2.350457,1387.60,Monitor / Potential Sell,0.81,Monitor / Potential Sell,AI-guided recommendation
2,CSCO,CISCO SYS INC,12.612230,3.743042,1379.16,Hold,0.72,Hold,AI-guided recommendation
3,NRC,NTNL RESH CORP,10.298946,-0.106440,1126.20,Monitor / Potential Sell,0.90,Monitor / Potential Sell,AI-guided recommendation
4,LLY,ELI LILLY AND CO,9.827802,11.187212,1074.68,Hold,0.92,Hold,AI-guided recommendation
5,WBD,WARNER BROS DISCOVERY INTENDER OFFER EXP: 01/2...,8.170204,134.057269,893.42,Hold / Consider Taking Profit,0.94,Consider Taking Profit,High gain (+134.1%)
6,BTE,BAYTEX ENERGY CORP F,6.498340,-1.223242,710.60,Monitor / Potential Sell,0.95,Monitor / Potential Sell,AI-guided recommendation
7,COHR,COHERENT CORP,5.063603,89.050497,553.71,Hold / Consider Taking Profit,0.83,Consider Taking Profit,High gain (+89.1%)
8,VICR,VICOR CORP,3.006831,11.711344,328.80,Hold,0.77,Hold,AI-guided recommendation
9,ANRO,ALTO NEUROSCIENCE IN,2.767236,-0.669643,302.60,Monitor / Potential Sell,0.86,Monitor / Potential Sell,AI-guided recommendation


✅ Portfolio report saved: Portfolio_Reports/Portfolio_Report_2026-01-02_00-55-59.html
Click the link below to download the HTML report:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ **What This Section Does**

1. Interactive HTML Charts

- Each Plotly chart is exported to its own HTML file

- Embedded via `<iframe>` in the final report
<br>
2. Summary Table

- Includes allocation, gain/loss %, AI recommendations, confidence, rebalancing suggestion, and reasoning

- Styled with striped hoverable rows for readability
<br>
3. Reusable & Timestamped

- Reports are saved with a timestamp

- Can run daily or weekly for automated updates
<br>
4. Production-ready Workflow

- Folders automatically created

- All outputs in a single, professional HTML report

In [17]:
# ==============================
# Full Professional Portfolio Reporting with Conditional Formatting
# ==============================

from IPython.display import display, HTML
from google.colab import files
import plotly.io as pio
from datetime import datetime
import os

# ------------------------------
# Step 0: Create folder for reports
# ------------------------------
report_folder = "Portfolio_Reports"
os.makedirs(report_folder, exist_ok=True)

# ------------------------------
# Step 1: Display Charts Inline
# ------------------------------
print("### Portfolio Allocation ###")
fig_allocation.show()

print("### AI-Assisted Recommendations ###")
fig_recommendations.show()

print("### Multi-Step Rebalancing Suggestions ###")
fig_rebalancing.show()

# ------------------------------
# Step 2: Display Summary Table with Conditional Formatting Inline
# ------------------------------
def recommendation_color_html(action):
    colors = {
        "Reduce Position": "#FF6F61",             # Red
        "Monitor / Consider Sell": "#FFD700",     # Yellow
        "Consider Taking Profit": "#FFA500",      # Orange
        "Hold": "#4CAF50",                        # Green
        "Hold / Limit Exposure": "#4CAF50"        # Green
    }
    return colors.get(action, "#B0B0B0")          # Default gray

# Generate HTML for inline display
table_html_inline = "<table class='table table-striped table-hover'>"
table_html_inline += "<tr><th>Symbol</th><th>Name</th><th>Allocation %</th><th>Gain/Loss %</th><th>Market Value</th><th>AI Recommendation</th><th>Confidence</th><th>Rebalancing Suggestion</th><th>Reason</th></tr>"

for idx, row in portfolio_df.iterrows():
    color = recommendation_color_html(row['Rebalancing_Suggestion'])
    table_html_inline += f"<tr>"
    table_html_inline += f"<td>{row['Symbol']}</td>"
    table_html_inline += f"<td>{row['Name']}</td>"
    table_html_inline += f"<td>{row['Allocation_%']:.2f}</td>"
    table_html_inline += f"<td>{row['Gain_Loss_%']:.2f}</td>"
    table_html_inline += f"<td>${row['Market_Value']:,.2f}</td>"
    table_html_inline += f"<td>{row['AI_Recommendation']}</td>"
    table_html_inline += f"<td>{row['Confidence']:.2f}</td>"
    table_html_inline += f"<td style='background-color:{color}; color:white;'>{row['Rebalancing_Suggestion']}</td>"
    table_html_inline += f"<td>{row['Reason']}</td>"
    table_html_inline += f"</tr>"

table_html_inline += "</table>"

print("### Portfolio Summary Table with Conditional Formatting ###")
display(HTML(table_html_inline))

# ------------------------------
# Step 3: Save Full HTML Report with Conditional Formatting
# ------------------------------
# Save charts separately
pio.write_html(fig_allocation, file=os.path.join(report_folder, "Allocation_Chart.html"), auto_open=False)
pio.write_html(fig_recommendations, file=os.path.join(report_folder, "AI_Recommendations_Chart.html"), auto_open=False)
pio.write_html(fig_rebalancing, file=os.path.join(report_folder, "Rebalancing_Chart.html"), auto_open=False)

# Full HTML report
full_html = f"""
<html>
<head>
    <title>Portfolio Report - {datetime.now().strftime('%Y-%m-%d')}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 30px; }}
        h1 {{ color: #333333; }}
        h2 {{ color: #444444; margin-top: 30px; }}
        .table {{ border-collapse: collapse; width: 100%; }}
        .table th, .table td {{ border: 1px solid #ddd; padding: 8px; text-align: center; }}
        .table th {{ background-color: #f2f2f2; }}
    </style>
</head>
<body>
    <h1>Gemini Integrated Portfolio Insights</h1>
    <h2>Portfolio Summary Table</h2>
    {table_html_inline}
    <p>Interactive charts are saved separately in the folder: {report_folder}</p>
</body>
</html>
"""

# Save the report
report_filename = f"Portfolio_Report_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.html"
report_path = os.path.join(report_folder, report_filename)
with open(report_path, "w") as f:
    f.write(full_html)

# ------------------------------
# Step 4: Download Link
# ------------------------------
print(f"✅ Portfolio report with conditional formatting saved: {report_path}")
print("Click the link below to download the HTML report:")
files.download(report_path)

### Portfolio Allocation ###


### AI-Assisted Recommendations ###


### Multi-Step Rebalancing Suggestions ###


### Portfolio Summary Table with Conditional Formatting ###


Symbol,Name,Allocation %,Gain/Loss %,Market Value,AI Recommendation,Confidence,Rebalancing Suggestion,Reason
WDC,WESTERN DIGITAL CORP,17.33,11.32,"$1,894.97",Hold,0.81,Hold,AI-guided recommendation
FLS,FLOWSERVE CORP,12.69,-2.35,"$1,387.60",Monitor / Potential Sell,0.81,Monitor / Potential Sell,AI-guided recommendation
CSCO,CISCO SYS INC,12.61,3.74,"$1,379.16",Hold,0.72,Hold,AI-guided recommendation
NRC,NTNL RESH CORP,10.30,-0.11,"$1,126.20",Monitor / Potential Sell,0.90,Monitor / Potential Sell,AI-guided recommendation
LLY,ELI LILLY AND CO,9.83,11.19,"$1,074.68",Hold,0.92,Hold,AI-guided recommendation
WBD,WARNER BROS DISCOVERY INTENDER OFFER EXP: 01/21/26,8.17,134.06,$893.42,Hold / Consider Taking Profit,0.94,Consider Taking Profit,High gain (+134.1%)
BTE,BAYTEX ENERGY CORP F,6.50,-1.22,$710.60,Monitor / Potential Sell,0.95,Monitor / Potential Sell,AI-guided recommendation
COHR,COHERENT CORP,5.06,89.05,$553.71,Hold / Consider Taking Profit,0.83,Consider Taking Profit,High gain (+89.1%)
VICR,VICOR CORP,3.01,11.71,$328.80,Hold,0.77,Hold,AI-guided recommendation
ANRO,ALTO NEUROSCIENCE IN,2.77,-0.67,$302.60,Monitor / Potential Sell,0.86,Monitor / Potential Sell,AI-guided recommendation


✅ Portfolio report with conditional formatting saved: Portfolio_Reports/Portfolio_Report_2026-01-02_00-58-25.html
Click the link below to download the HTML report:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ **What’s New**

1. Conditional coloring of Rebalancing Suggestion column

- Makes key actions instantly visible to a viewer

- Red = urgent, Orange/Yellow = moderate, Green = safe

2. HTML report still Colab-friendly

- Inline charts remain interactive

- Table is now visually clear and professional

3. Ready for portfolio presentation

- Perfect for showing AI + engineering + financial skills in portfolio

# Gemini Integrated Portfolio Insights

### Purpose
This notebook demonstrates a **comprehensive, AI-assisted portfolio analysis pipeline**. It is designed to provide **investors and analysts with actionable insights** while showcasing advanced **data engineering, visualization, and AI reasoning skills**.

---

### Workflow Overview
1. **Portfolio Data Loading & Cleaning**  
   - Load current portfolio positions with market value, allocation %, gain/loss %, and cost basis.  
   - Prepare data for analysis and visualization.

2. **Portfolio Visualization**  
   - Interactive **allocation donut chart**.  
   - **Gain/Loss scatter plots** and **bubble charts** for AI-assisted recommendations.  
   - Multi-step **rebalancing suggestion scatter chart**.

3. **AI-Assisted Insights**  
   - Detect high concentration risk (>20% allocation).  
   - Identify top performers and underperformers.  
   - Generate actionable AI recommendations with **confidence scores**.  

4. **Multi-Step Reasoning & Rule-Grounded Rebalancing**  
   - Combine AI recommendations with simple financial rules:  
     - Reduce positions with high allocation  
     - Monitor or consider selling underperformers  
     - Consider taking profit on extreme gains  
     - Limit exposure for positions >25% of portfolio  

5. **Professional Reporting**  
   - Inline, interactive charts in Colab.  
   - Summary table with **conditional formatting** for clear visualization of recommendations.  
   - Export **downloadable HTML report** for offline review and presentation.  

---

### Skills Demonstrated
- **Data Engineering & Cleaning**: Transforming portfolio data for analysis.  
- **Interactive Visualization**: Plotly charts with dynamic sizing and color coding.  
- **Applied AI & Multi-Step Reasoning**: Generating actionable portfolio insights with confidence-aware recommendations.  
- **Financial Logic Integration**: Combining AI output with portfolio risk and rebalancing rules.  
- **Reporting & Automation**: Conditional formatting, HTML export, and downloadable interactive reports.  

---

This notebook is designed as a **portfolio-ready demonstration** of full-stack **data, AI, and finance engineering capabilities** suitable for potential employers or investors.